In [3]:
import osmnx as ox
import geopandas as gpd
import folium
import requests
import random

TREFLE_API_KEY = "usr-53OJoVJelq6XE2f6FFcc8dxujBkvuNd7VRlUUkkEf2g"
PLANT_ID_API_KEY = "vAJN47WND9UCBn4scfm6OymPRfWGUnDWbJA6MciHXSvZYrvEyq"

USE_STREET_VIEW_TREES = False  # Set True to use street-view data instead of OSM

# If using street-view, provide a GeoDataFrame `street_view_trees` with Point geometries
# Example (replace with actual street-view tree detection results):
# street_view_trees = gpd.read_file("/notebooks/street_view_trees.geojson")

# Define TTDI bounding box
north, south, east, west = 3.1505, 3.128, 101.641, 101.615
bbox = (west, south, east, north)

# if USE_STREET_VIEW_TREES:
#     print("Using street-view detected trees")
#     try:
#         existing_trees = street_view_trees.to_crs(epsg=3857)
#         existing_trees = existing_trees[existing_trees.geometry.type == 'Point']
#         print(f"Street-view trees found: {len(existing_trees)}")
#     except Exception as e:
#         print(f"Error loading street-view trees: {e}")
#         existing_trees = gpd.GeoDataFrame(columns=['geometry'], geometry=[])
# else:
#     # Fallback to OSM trees
tree_tags = {'natural': 'tree'}
try:
    existing_trees = ox.features_from_bbox(bbox, tree_tags)
    existing_trees = existing_trees.to_crs(epsg=3857)
    existing_trees = existing_trees[existing_trees.geometry.notnull()]
    existing_trees = existing_trees[existing_trees.geometry.type == 'Point']
    print(f"Existing trees (OSM) found: {len(existing_trees)}")
except Exception as e:
    print(f"Error fetching existing trees: {e}")
    existing_trees = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# Fetch plantable areas (parks, grass, meadows)
tags = {
    'leisure': 'park',
    'landuse': 'grass',
    'natural': 'grass'
}

try:
    plantable_areas = ox.features_from_bbox(bbox, tags)
    plantable_areas = plantable_areas.to_crs(epsg=3857)
    plantable_areas = plantable_areas[plantable_areas.geometry.notnull()]
    # Explode mixed geometries
    plantable_areas = plantable_areas.explode(index_parts=False).reset_index(drop=True)
    plantable_areas = plantable_areas[plantable_areas.geometry.type.isin(['Polygon', 'MultiPolygon'])]
    print(f"Plantable areas found: {len(plantable_areas)}")
except Exception as e:
    print(f"Error fetching plantable areas: {e}")
    plantable_areas = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# Fetch existing trees from OSM
tree_tags = {'natural': 'tree'}
try:
    existing_trees = ox.features_from_bbox(bbox, tree_tags)
    existing_trees = existing_trees.to_crs(epsg=3857)
    existing_trees = existing_trees[existing_trees.geometry.notnull()]
    existing_trees = existing_trees[existing_trees.geometry.type == 'Point']
    print(f"Existing trees found: {len(existing_trees)}")
except Exception as e:
    print(f"Error fetching existing trees: {e}")
    existing_trees = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# Remove areas too close to existing trees
if not existing_trees.empty and not plantable_areas.empty:
    existing_trees['geometry'] = existing_trees.geometry.buffer(2)  # 2m buffer
    plantable_areas = gpd.overlay(plantable_areas, existing_trees, how='difference')
    print(f"Plantable areas after removing existing trees: {len(plantable_areas)}")
    

def suggest_tree(area_m2: float, context="urban Malaysia"):
    """
    Suggests a tropical tree species based on area size,
    fetching detailed info from Plant.id.
    """

    # Size brackets
    if area_m2 < 5:
        size_category = "tiny"
    elif area_m2 < 50:
        size_category = "small"
    elif area_m2 < 500:
        size_category = "medium"
    elif area_m2 < 5000:
        size_category = "large"
    else:
        size_category = "very_large"

    # Fallback
    local_fallback = {
        "tiny": "Hibiscus",
        "small": "Ixora",
        "medium": "Plumeria",
        "large": "Angsana",
        "very_large": "Rain Tree"
    }

    # Keywords for Trefle search
    trefle_terms = {
        "tiny": ["shrub", "succulent"],
        "small": ["tropical shrub", "ornamental bush"],
        "medium": ["urban tree", "tropical flowering tree"],
        "large": ["shade tree", "tropical hardwood"],
        "very_large": ["rain tree", "dipterocarpus", "canopy tree"]
    }

    # Pick a random search term
    query = random.choice(trefle_terms[size_category])

    # Get tree name from Trefle ---
    trefle_url = f"https://trefle.io/api/v1/plants/search?q={query}&token={TREFLE_API_KEY}"
    try:
        trefle_resp = requests.get(trefle_url, timeout=10)
        trefle_data = trefle_resp.json()
        if "data" in trefle_data and len(trefle_data["data"]) > 0:
            trefle_choice = random.choice(trefle_data["data"])
            tree_name = trefle_choice.get("common_name") or trefle_choice.get("scientific_name")
        else:
            tree_name = local_fallback[size_category]
    except Exception:
        tree_name = local_fallback[size_category]

    # Get description from Plant.id
    description = "Description not found."
    try:
        headers = {"Api-Key": PLANT_ID_API_KEY, "Content-Type": "application/json"}
        search_url = f"https://plant.id/api/v3/kb/plants/name_search?q={tree_name}"
        search_resp = requests.get(search_url, headers=headers, timeout=10)
        data = search_resp.json()
        entities = data.get("entities", [])
        if entities:
            access_token = entities[0].get("access_token")
            detail_url = f"https://plant.id/api/v3/kb/plants/{access_token}?details=description"
            detail_resp = requests.get(detail_url, headers=headers, timeout=10)
            detail_data = detail_resp.json()
            description = detail_data.get("description", {}).get("value", "Description not available.")
    except Exception as e:
        description = f"No description available (error: {e})"

    # Result
    reason = f"Selected as a suitable {size_category.replace('_', ' ')} tree for {context} ({area_m2:.1f} m²)."

    return {
        "tree_name": tree_name,
        "reason": reason,
        "description": description
    }

def add_tree_popup(map_obj, geometry, area_m2, tree_info):
    """
    Adds a popup with full details for the suggested tree.
    """
    html_content = f"""
    <div style='font-size:13px; max-width: 350px; overflow-y:auto; max-height: 300px;'>
        <b>Area:</b> {area_m2:.1f} m²<br>
        <b>Suggested Tree:</b> {tree_info.get('tree_name', 'Unknown')}<br>
        <b>Reason:</b> {tree_info.get('reason', 'N/A')}<br>
        <b>Scientific Name:</b> {tree_info.get('scientific_name', '')}<br>
        <b>Description:</b><br>{tree_info.get('description', 'No details.')}
    </div>
    """

    folium.GeoJson(
        geometry,
        style_function=lambda feature, color="blue": {
            'fillColor': color,
            'color': 'darkblue',
            'weight': 1,
            'fillOpacity': 0.5
        },
        popup=folium.Popup(html_content, max_width=400)
    ).add_to(map_obj)

# Calculate plot areas and suggest trees
plantable_areas['area_m2'] = plantable_areas.geometry.area
plantable_areas['suggested_tree'] = plantable_areas['area_m2'].apply(lambda a: suggest_tree(a))

# Convert to WGS84 for Folium
plantable_areas = plantable_areas.to_crs(epsg=4326)
existing_trees = existing_trees.to_crs(epsg=4326)

center_lat = (north + south) / 2
center_lon = (east + west) / 2

# Create Folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=16, tiles='CartoDB positron')

# Shade plantable areas
for _, row in plantable_areas.iterrows():
    add_tree_popup(m, row.geometry, row["area_m2"], row["suggested_tree"])

# Shade existing tree areas (green)
for _, row in existing_trees.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda feature: {
            "fillColor": "green",
            "color": "green",
            "weight": 1,
            "fillOpacity": 0.5
        },
        tooltip="Existing tree"
    ).add_to(m)

m


Existing trees (OSM) found: 12
Plantable areas found: 23
Existing trees found: 12
Plantable areas after removing existing trees: 23
